# MegaDetector

In [2]:
import os
import json
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
from PytorchWildlife.models import detection as pw_detection

# Path to the MegaDetector code directory #No longer used
md_code_path = "CameraTraps/detection"

# Path to the model we downloaded #No longer used
model_path = "md_v5a.0.pb"

In [4]:
# Input image path
image_path = 'D:/orinoquia_camera_traps_images/A06/100EK113/01210093.JPG'
# Output path for the image with bounding box
output_path = "detection_output.jpg"

# Function to run MegaDetector on a single image
def run_megadetector_on_image(image_path):
    detection_model = pw_detection.MegaDetectorV6(version="MDV6-yolov9-c") # Model weights are automatically downloaded.

    img = np.array(Image.open(image_path))
    detection_results = detection_model.single_image_detection(img)

    detections = {}
    detections["images"] = {}
    detections["images"][image_path] = []

    # Format the results as MegaDetector output
    if len(detection_results) > 0: # Make sure there are not detections, to avoid errors
        boxes, scores, classes = detection_results
        for bbox, confidence, category in zip(boxes, scores, classes):
            try:
                bbox_list = bbox.tolist()
            except AttributeError as e:
                print(f"Error converting bounding box: {e}")
                print(f"Type of bbox: {type(bbox)}")  # See the type of object bbox
                bbox_list = bbox # Keep the object "as is" 

            # convert confidence score for correct output
            try:
                confidence_val = float(confidence) # Convert confidence score
            except ValueError as e:
                 print(f"Could not convert confidence score {confidence} to float") #See the value of confidence
                 confidence_val = 0 # Fallback value, change to appropriate score

            detections["images"][image_path].append({
                "bbox": bbox_list,
                "confidence": confidence_val,
                "category": str(category.item()), # if category is a tensor
            })

    return detections

# Helper function to draw bounding boxes on images
def draw_boxes(image_path, detections, output_path):
    img = np.array(Image.open(image_path))

    fig, ax = plt.subplots(1)
    ax.imshow(img)
    # Plot the bounding boxes
    if(image_path in detections["images"]):
        for detection in detections["images"][image_path]:
              bbox = detection['bbox']
              confidence = detection['confidence']
              # Scale bounding box coordinates from [y1, x1, y2, x2] to image size
              height, width, _ = img.shape
              x1 = int(bbox[1] * width)
              y1 = int(bbox[0] * height)
              x2 = int(bbox[3] * width)
              y2 = int(bbox[2] * height)
              rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor='r', facecolor='none')
              ax.add_patch(rect)
              ax.text(x1, y1 - 5, f'{confidence:.2f}', color='white', fontsize=8, backgroundcolor='r')
    plt.savefig(output_path)
    plt.close(fig)

# Run MegaDetector on the image
detections = run_megadetector_on_image(image_path)

# Draw bounding boxes and save the output image
draw_boxes(image_path, detections, output_path)

print(f"Saved detection to {output_path}")

Ultralytics 8.3.74  Python-3.11.2 torch-2.6.0+cpu CPU (Intel Core(TM) i7-10750H 2.60GHz)
YOLOv9c summary (fused): 384 layers, 25,321,561 parameters, 0 gradients, 102.3 GFLOPs

0: 480x640 1 animal, 503.6ms
Speed: 5.0ms preprocess, 503.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Error converting bounding box: 'str' object has no attribute 'tolist'
Type of bbox: <class 'str'>
Could not convert confidence score d to float


AttributeError: 'str' object has no attribute 'item'